In [19]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 加载数据

In [20]:
# step1 加载文件
with open('../../data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 使用BERT-Base-Chinese

In [21]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese
embeddings = np.load('../../data/embedding_bbc.npy') # 使用bert-base-chinese向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=10,
  min_samples=5,
  metric='euclidean'
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

# 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

# 查看主题
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_model.get_topic_info()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,326,-1_景区_河南_城市_中国,"[景区, 河南, 城市, 中国, 游客, 遗址, 博物馆, 历史, 洛阳市, 活动]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,98,0_活动_免费_景区_高速,"[活动, 免费, 景区, 高速, 门票, 时间, 白云山, 地点, 栾川, 安全可靠]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
2,1,52,1_晚会_广场_活动_文旅,"[晚会, 广场, 活动, 文旅, 体验, 话题, 沉浸, 街头, 历史, 隋唐洛阳城]",[中国 旅游 泉州 举行 多项 文旅 活动 鲤城区 城南 庙会 再次 开启 传统工艺 传统 ...
3,2,44,2_建设_机场_项目_规划,"[建设, 机场, 项目, 规划, 铁路, 自驾车, 黄河, 航线, 国家, 发展]",[国家 发展 改革 印发 汉江 生态 经济带 发展 规划 通知 汉江 生态 经济带 规划 范...
4,3,42,3_地方_很多_一天_一点,"[地方, 很多, 一天, 一点, 已经, 时间, 古城, 酒店, 可惜, 西安]",[洛阳 值得 一去 地方 简单 下来 洛阳 旅游 攻略 建议 夏天 洛阳 洛阳 市区 主要 ...
5,4,42,4_泉州_洛阳桥_旅行_历史,"[泉州, 洛阳桥, 旅行, 历史, 济南, 沧州, 黄陂, 中国, 古城, 大福]",[海丝 泉州 夏天 清晨 洛阳桥 洛阳 日出 夏天 清晨 来到 泉州 洛阳桥 沐浴 晨光 长...
6,5,31,5_竹海_王府_河南_旅游区,"[竹海, 王府, 河南, 旅游区, 游泳, 处理, 栾川, 夏天, 水席, 景区]",[没错 没错 最近 夏日 火热 潮流 汉服 悄然 洛阳 山水 景区 吸引 大量 拍照 达人 ...
7,6,30,6_石窟_龙门石窟_艺术_中国,"[石窟, 龙门石窟, 艺术, 中国, 朝代, 世界, 造像, 文化遗产, 宝库, 莫高窟]",[洛阳 周边游 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护...
8,7,27,7_游客_午餐_老君山_景区,"[游客, 午餐, 老君山, 景区, 表示, 孙军范, 卷入, 同学, 工作人员, 网友]",[无人 值守 一元 午餐 结账 多出 国庆 假期 景区 连续 推出 午餐 共售 游客 点赞 ...
9,8,27,8_防控_疫情_景区_开放,"[防控, 疫情, 景区, 开放, 场所, 关闭, 恢复, 人员, 娱乐场所, 落实]",[洛阳 即日起 洛阳 暂时 关闭 网吧 KTV 人员 聚集 密闭 场所 小编 刚刚 洛阳市 ...


In [22]:
# ⭐ 打印主题信息
topic_docs = topic_model.get_document_info(docs)
topic_docs.to_csv('./聚类结果.csv')

In [23]:
with open('../../data/文本.txt', 'r', encoding='utf-8') as file:
  texts = file.readlines()
  print('文本条数：', len(texts))
  topic_docs.insert(1, '原文', texts)
with open('../../data/时间.txt', 'r', encoding='utf-8') as file:
  years = file.readlines()
  print('文本条数：', len(years))
  topic_docs.insert(2, '时间', years)
topic_docs.to_csv('./聚类结果2.csv')


文本条数： 1000
文本条数： 1000
